In [434]:
from crawler import download
from bs4 import BeautifulSoup
import bs4
import requests
import time
import re
import sqlite3
from html import unescape
import json
import sys
from collections import defaultdict
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

In [5]:
def parseURL(seed):
    html = download("get", seed)
    dom = BeautifulSoup(html.text, "html.parser")

    return [
        requests.compat.urljoin(seed,
                                tag.find_parent()["href"])
        for tag in dom.select("font.list_title")
        if tag.find_parent() and tag.find_parent().has_attr("href")
    ]

def parseContent(url):
    html = download("get", url)
    dom = BeautifulSoup(html.text, "html.parser")

    return {
        "title":
        dom.select_one("font.view_title2").text.strip(),
        "body":
        dom.select_one("table.pic_bg td.han").text.strip(),
        "comments": [
            tag.text.strip("table.pic_bg td.han")
            for tag in dom.select("div[id^=commentContent_]")
        ]
    }

In [437]:
def waitUntil(cssSelect=None, maximum=10, interval=1):
    try:
        WebDriverWait(driver, maximum, interval,
                     [ElementNotInteractableException]).until(EC.visibility_of_element_located(
        (By.CSS_SELECTOR, cssSelect)))
    except:
        return False
    
    return True

In [351]:
driver = webdriver.Chrome()

In [352]:
driver.get("http://ticket.interpark.com/")

In [81]:
_logout = [_["name"] for _ in driver.get_cookies()]
#_logout

['_gat_UA-93889457-1',
 '_gid',
 '_ga',
 '_trs_id',
 'cto_lwid',
 '_trs_sid',
 '_fbp',
 'criteo_write_test',
 'OAX',
 '_trs_flow',
 'pcid',
 'ASPSESSIONIDQADTCTSQ',
 'ASPSESSIONIDSCCTDSRQ']

In [65]:
%%writefile interpark.json
{"id": "pje49990", "pw": "tlqdlqjs12"}

Overwriting interpark.json


In [66]:
with open("interpark.json") as fp:
    account = json.load(fp)

login = driver.find_element_by_css_selector("#logstatus > a")
if login and login.get_attribute("id") == "aLogin":
    driver.get(login.get_attribute("href"))
    driver.find_element_by_css_selector("#UID").send_keys(account["id"])
    driver.find_element_by_css_selector("#PWD").send_keys(account["pw"])
    driver.find_element_by_css_selector("div.loginBtn > button.btnRed").click()
else :
    print("로그인 됨")

이걸 함수로 만든것!

In [140]:
# 로그인 버튼 찾기
def login(account, method="login"):
    try:
        login = driver.find_element_by_css_selector("#logstatus > a")
        if method == "login":
            if login and login.get_attribute("id") == "aLogin":
                driver.get(login.get_attribute("href"))
                driver.find_element_by_css_selector("#UID").send_keys(account["id"])
                driver.find_element_by_css_selector("#PWD").send_keys(account["pw"])
                driver.find_element_by_css_selector("div.loginBtn > button").click()
        else:
            login.click()
    except:
        return False
    return True

In [253]:
def safeClose():
    if len(driver.window_handles) > 1:
        for _ in driver.window_handles[1:]:
            driver.switch_to_window(_)
            driver.close()
        driver.switch_to_window(driver.window_handles[0])
        driver.switch_to_default_content()

In [141]:
login(account,"login")

True

In [142]:
_login = [_["name"] for _ in driver.get_cookies()]
#_login

In [143]:
set(_login) - set(_logout)

{'ASP.NET_SessionId',
 'ASPSESSIONIDQCDSDRRQ',
 'ASPSESSIONIDSCBQBQQR',
 'ASPSESSIONIDSCBSASRQ',
 'GNBBookCnt',
 'GoodsCntInWishAndCart',
 'HMem%5FNO',
 'IEGS%5FGUIF',
 'IMem%5FNO',
 'ISession%5FID',
 'MemHash',
 'PrivacyCertify',
 'SAVEID',
 'SnsState',
 'TKLogin',
 'TMem%5FNO',
 'TPartner',
 'TSecLogin',
 'TSession%5FID',
 'alMemNm',
 'captcha_text',
 'cpth',
 'g_buyAmt',
 'g_buyCnt',
 'g_memGrd',
 'iaf',
 'index_popup',
 'interparkID',
 'interparkSNO',
 'interparkstamp',
 'loginRetUrl',
 'loginSnsTp',
 'loginTp',
 'memRealName',
 'pwdChgYn',
 'reqSite',
 'siteAgree',
 'smsCertYN',
 'tempinterparkGUEST',
 'tp'}

이 친구는 로그아웃 되어 있으면 로그인하고, 로그인 되어 있으면 로그아웃 하는 것

In [346]:
if "ISession%5FID" not in [_["name"] for _ in driver.get_cookies()]:
    login(account)
else:
    login(None, "logout")

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=75.0.3770.142)


In [353]:
links = [_ for _ in driver.find_elements_by_css_selector("ul.gnbBtns a[class^=btn]")]
[_.find_element_by_css_selector("img").get_attribute("alt") for _ in links]

['전체메뉴보기',
 '뮤지컬',
 '콘서트',
 '연극',
 '클래식/무용',
 '스포츠',
 '레저',
 '전시/행사',
 '아동/가족',
 'TOPING',
 '랭킹',
 '지역',
 '공연장']

In [354]:
def gnbbtns(category="콘서트"):
    links = [_ for _ in driver.find_elements_by_css_selector("ul.gnbBtns a[class^=btn]")]
    titles = [_.find_element_by_css_selector("img").get_attribute("alt") for _ in links]
    
    if category in titles:
        return links[titles.index(category)].get_attribute("href")
    return False

In [355]:
gnbbtns("콘서트")

'http://ticket.interpark.com/ConcertIndex.asp'

In [356]:
url = gnbbtns("콘서트")
if url:
    driver.get(url)

In [343]:
def hotissue(issue=""):
    issues = [_ for _ in driver.find_elements_by_css_selector(
        ".wrap_issue dl.list > dt.issue_obj > a")]
    titles = [_.find_element_by_css_selector("span.txt1").text for _ in issues]
    ##span.txt1이 가수이름!!
    
    if issue in titles:
        return issues[titles.index(issue)].get_attribute("href")
    else:
        return issues[0].get_attribute("href") 

In [357]:
hotissue("박정현")

'http://ticket.interpark.com/Ticket/Goods/TPBridge.asp?GoodsCode=19008643'

In [358]:
url = hotissue("박정현")
if url:
    driver.get(url)

In [274]:
def bookWindow(options=None):
    if options:
        iframe = driver.find_element_by_css_selector(
            "#ifrCalendar")
        
        if iframe:
            driver.switch_to_frame(iframe)
            dates = [_ for _ in
                    driver.find_elements_by_css_selector(
                        "td.able > a")]
            title = [_.text for _ in dates]
            
        if options in title:
            driver.execute_script(
                dates[title.index(options)]
                .get_attribute("onclick"))
        
        else:
            driver.execute_script(
                dates[0].get_attribute("onclick"))
        
        driver.switch_to_default_content()
        
    driver.find_element_by_css_selector(
        ".tk_dt_btn_TArea > a.btn_rev").click()
    
    driver.switch_to_window(driver.window_handles[-1])
    driver.find_element_by_css_selector("#LargeNextBtn").click()
    

In [359]:
bookWindow("4")

C:\Users\Ikjong\Anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: use driver.switch_to.frame instead
  import sys
C:\Users\Ikjong\Anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: use driver.switch_to.default_content instead
C:\Users\Ikjong\Anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:27: DeprecationWarning: use driver.switch_to.window instead


## 좌석선택

In [436]:
def step2(floor=None, section=None, row=None, seat = []):
    iframe1 = driver.find_element_by_css_selector("#ifrmSeat")
    driver.switch_to_frame(irame1)
    
    driver.execute_script("fnRefresh();")
    
    if floor:
        iframe2 = driver.find_element_by_css_selector("#ifrmSeatView")
        driver.switch_to_frame(iframe2)
        area = [_ for _ in driver.find_element_by_css_selector("map > area")]
        name = [re.findall(r"(RGN\d{3})",_.get_attribute("href"))[0] for _ in area]
        
        floor = "RGN{:03d}".format(floor)
        
        if floor in name:
            driver.execute_script(area[name.index(floor)].get_attribute("href"))
        
        driver.switch_to_default_content()
        driver.switch_to_frame(iframe1)
    
    iframe3 = driver.find_element_by_css_selector("#ifrmSeatDetail")
    driver.switch_to_frame(iframe3)
    
    seatList = driver.find_element_by_css_selector("#TmgsTable img.stySeat")
    seatDict = defaultdict(lambda: defaultdict(list))
    
    for i, _ in enumerate(seatList):
        script = re.findall(
        r"\(\'.*?\', \'.*?\', \'.*?\', \'(.*?)\', \'(.*?)\', \'.*?\'", _.get_attribute("onclick"))[0]
        
        seatInfo = script[0].split()
        if len(seatInfo) < 2:
            seatDict["2층"][seatInfo[0]].append(
                (script[1], i))
        else :
            seatDict[seatInfo[0]][seatInfo[1]].append(
                (script[1], i))
            
    if not section:
        section = sorted(seatDict.keys())[0]
    
    if not row:
        row = sorted(seatDict[section].keys())[0]
    
    if len(seat) > 1:
        selected = [_ for _ in seatDict[section][row]
                       if _[0] in seat]
    else :
        selected = [_ for _ in sorted(seatDict[section][row])]
    print(selected)
    
    for _ in selected:
        driber.execute_script(
        seatList[_[1]].get_attribute("onclick"))
        
    driver.switch_to_default_content()

In [333]:
def book_seat_Window(options=None):
    
    
    iframe = driver.find_element_by_name("ifrmSeat")
    iiframe = driver.find_element_by_name("ifrmSeatDetail")
    driver.switch_to_frame(iiframe)
    script = driver.find_elements_by_css_selector("tbody > tr > td > img.stySeat")[1].get_attribute("onclick")
    driver.execute_script(script)
    
    driver.switch_to_default_content()
    iframe = driver.find_element_by_name("ifrmSeat")
    driver.switch_to_frame(iframe)
    click_script = driver.find_element_by_css_selector("div.btnWrap > a").get_attribute("onclick")
    driver.execute_script(click_script)
    driver.switch_to_default_content()


In [276]:
driver.switch_to_window(driver.window_handles[-1])

C:\Users\Ikjong\Anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.window instead
  """Entry point for launching an IPython kernel.


In [278]:
driver.find_element_by_css_selector("#LargeNextBtn").click()

In [362]:
iframe = driver.find_element_by_name("ifrmSeat")

In [363]:
driver.switch_to_frame(iframe)

C:\Users\Ikjong\Anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.frame instead
  """Entry point for launching an IPython kernel.


In [364]:
iiframe = driver.find_element_by_name("ifrmSeatDetail")

In [365]:
driver.switch_to_frame(iiframe)

C:\Users\Ikjong\Anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.frame instead
  """Entry point for launching an IPython kernel.


In [366]:
script = driver.find_elements_by_css_selector("tbody > tr > td > img.stySeat")[1].get_attribute("onclick")

In [367]:
driver.execute_script(script)

In [368]:
driver.switch_to_default_content()

C:\Users\Ikjong\Anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.default_content instead
  """Entry point for launching an IPython kernel.


In [369]:
iframe = driver.find_element_by_name("ifrmSeat")

In [370]:
driver.switch_to_frame(iframe)

C:\Users\Ikjong\Anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.frame instead
  """Entry point for launching an IPython kernel.


In [371]:
driver.find_element_by_css_selector("div.btnWrap > a")

<selenium.webdriver.remote.webelement.WebElement (session="776981614e398521ebdbc5acb155d99a", element="b2da5760-8cff-4520-a1ac-fc6f5d6e99bc")>

In [372]:
click_script = driver.find_element_by_css_selector("div.btnWrap > a").get_attribute("onclick")

In [373]:
driver.execute_script(click_script)

In [374]:
driver.switch_to_default_content()

C:\Users\Ikjong\Anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.default_content instead
  """Entry point for launching an IPython kernel.


## 여기서 매수 선택하기

In [399]:
from selenium.webdriver.support.ui import Select

In [400]:
def setPrice():
    if not waitUntil("#ifrmBookStep"):
        print("Not found Book Frame")
        return False
    
    driver.switch_to_frame(
        driver.find_element_by_css_selector("ifrmBookStep"))
    
    if not waitUntil("select[name='SeatCount']"):
        print("Not found Seat Count")
        return False
    
    seatCount = Select(
        driver.find_element_by_css_selector(
            "select[name='SeatCount']"))
    seatCount.select_by_index(
        driver.find_element_by_css_selector(
            ".taL option:last-child").get_attribute("value"))
    
    driver.switch_to_default_content()

In [376]:
iframe = driver.find_element_by_name("ifrmBookStep")

In [377]:
driver.switch_to_frame(iframe)

C:\Users\Ikjong\Anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.frame instead
  """Entry point for launching an IPython kernel.


In [382]:
driver.find_elements_by_css_selector("select option")[1].click()

In [383]:
driver.switch_to_default_content()

C:\Users\Ikjong\Anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.default_content instead
  """Entry point for launching an IPython kernel.


In [392]:
NextStep = driver.find_elements_by_css_selector("#SmallNextBtn a")[1].get_attribute("href")

In [393]:
driver.execute_script(NextStep)

## 이제 개인정보 입력

In [395]:
iframe = driver.find_element_by_name("ifrmBookStep")
driver.switch_to_frame(iframe)

C:\Users\Ikjong\Anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: use driver.switch_to.frame instead
  


In [397]:
driver.find_element_by_css_selector("#YYMMDD").send_keys("960118")
driver.find_element_by_css_selector("#HpNo1").send_keys("010")
driver.find_element_by_css_selector("#HpNo2").send_keys("3357")
driver.find_element_by_css_selector("#HpNo3").send_keys("4990")

In [398]:
driver.switch_to_default_content()

C:\Users\Ikjong\Anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.default_content instead
  """Entry point for launching an IPython kernel.


In [402]:
NextStep = driver.find_elements_by_css_selector("#SmallNextBtn a")[1].get_attribute("href")
driver.execute_script(NextStep)

## 여기부터 결제

무통장 -> 결제은행 -> 결제 ... 주석버튼

In [415]:
iframe = driver.find_element_by_name("ifrmBookStep")
driver.switch_to_frame(iframe)

C:\Users\Ikjong\Anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: use driver.switch_to.frame instead
  


In [421]:
driver.find_element_by_css_selector("#Payment_22004 > td > input").click()

In [423]:
driver.find_element_by_css_selector("#BankCode > option:nth-child(7)").click()

In [424]:
driver.switch_to_default_content()
NextStep = driver.find_elements_by_css_selector("#SmallNextBtn a")[1].get_attribute("href")
driver.execute_script(NextStep)

C:\Users\Ikjong\Anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.default_content instead
  """Entry point for launching an IPython kernel.


In [426]:
iframe = driver.find_element_by_name("ifrmBookStep")
driver.switch_to_frame(iframe)

C:\Users\Ikjong\Anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: use driver.switch_to.frame instead
  


In [429]:
driver.find_element_by_css_selector("#CancelAgree").click()
driver.find_element_by_css_selector("#CancelAgree2").click()

In [431]:
driver.switch_to_default_content()
#driver.find_element_by_css_selector("#LargeNextBtnLink").click()

C:\Users\Ikjong\Anaconda3\envs\cuda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.default_content instead
  """Entry point for launching an IPython kernel.


In [433]:
safeClose()